In [39]:
import tensorflow as tf
import keras 
import numpy as np
from keras.models import Model, Sequential
from keras.layers import Flatten, Conv2D, MaxPooling2D, Dense, Input, Dropout, BatchNormalization, UpSampling2D, Conv2DTranspose, Reshape
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras import backend as k
import matplotlib.pyplot as plt

In [40]:
(train_x, train_y), (test_x, test_y) = tf.keras.datasets.mnist.load_data()

In [41]:
train_x[1].shape

(28, 28)

In [42]:
train_x.shape[1]

28

In [43]:
train_x.shape

(60000, 28, 28)

In [44]:
train_x = np.reshape(train_x, [-1, 28, 28, 1])
test_x = np.reshape(test_x, [-1, 28, 28, 1])

In [45]:
train_x = train_x.astype(float) / 255
test_x = test_x.astype(float) / 255

In [46]:
#train_y = to_categorical(train_y)
#test_y = to_categorical(test_y)

In [47]:
input_en = Input(shape=(28,28,1),name='input_en')
x = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', name='conv1')(input_en)
x = Conv2D(filters=512, kernel_size=5, padding='same', activation='relu', name='conv2')(x)
x = MaxPooling2D(name='pool1')(x)
x = Conv2D(filters=512, kernel_size=5, padding='same', activation='relu', name='conv3')(x)
x = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', name='conv4')(x)
x = MaxPooling2D(name='pool2')(x)
shape_en = k.int_shape(x)
x = Flatten()(x)
x = Dense(units=1024, activation='relu', name='dense1')(x)
x = Dense(units=512, activation='relu', name='dense2')(x)
output_en = Dense(units=100, activation='sigmoid',name='output_en')(x)
#shape_en = output_en.shape
model_en = Model(input=input_en, output=output_en, name='encoder_model')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="encoder_model", inputs=Tensor("in..., outputs=Tensor("ou...)`
  


In [48]:
output_en.shape

TensorShape([None, 100])

In [49]:
model_en.summary()

Model: "encoder_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_en (InputLayer)        (None, 28, 28, 1)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 28, 28, 64)        640       
_________________________________________________________________
conv2 (Conv2D)               (None, 28, 28, 512)       819712    
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 14, 14, 512)       0         
_________________________________________________________________
conv3 (Conv2D)               (None, 14, 14, 512)       6554112   
_________________________________________________________________
conv4 (Conv2D)               (None, 14, 14, 64)        294976    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 7, 7, 64)        

In [50]:
shape_en

(None, 7, 7, 64)

In [51]:
input_de = Input(shape=(100,),name='input_de')
x = Dense(units=512, activation='relu',name='denseT1')(input_de)
x = Dense(units=1024, activation='relu', name='denseT2')(x)
x = Dense((shape_en[1]*shape_en[2]*shape_en[3]), name='unknown')(x)
x = Reshape((shape_en[1], shape_en[2], shape_en[3]), name='Reshape_fn')(x)

x = UpSampling2D(name='upsampling1')(x)
x = Conv2DTranspose(filters=64, kernel_size=3, padding='same', activation='relu', name='conv2dT1')(x)
x = Conv2DTranspose(filters=512, kernel_size=5, padding='same', activation='relu', name='conv2dT2')(x)
x = UpSampling2D(name='upsampling2')(x)
x = Conv2DTranspose(filters=512, kernel_size=5, padding='same', activation='relu', name='conv2dT3')(x)
x = Conv2DTranspose(filters=64, kernel_size=3, padding='same', activation='relu', name='conv2dT4')(x)
output_de = Conv2DTranspose(filters=1, kernel_size=3, padding='same', activation='sigmoid', name='output_de')(x)
model_de = Model(input = input_de, output = output_de, name='model_decoder')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="model_decoder", inputs=Tensor("in..., outputs=Tensor("ou...)`
  


In [52]:
model_de.summary()

Model: "model_decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_de (InputLayer)        (None, 100)               0         
_________________________________________________________________
denseT1 (Dense)              (None, 512)               51712     
_________________________________________________________________
denseT2 (Dense)              (None, 1024)              525312    
_________________________________________________________________
unknown (Dense)              (None, 3136)              3214400   
_________________________________________________________________
Reshape_fn (Reshape)         (None, 7, 7, 64)          0         
_________________________________________________________________
upsampling1 (UpSampling2D)   (None, 14, 14, 64)        0         
_________________________________________________________________
conv2dT1 (Conv2DTranspose)   (None, 14, 14, 64)      

In [53]:
autoencoder = Model(input=input_en, output = model_de(model_en(input_en)), name='autoencoder')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="autoencoder", inputs=Tensor("in..., outputs=Tensor("mo...)`
  """Entry point for launching an IPython kernel.


In [54]:
autoencoder.summary()

Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_en (InputLayer)        (None, 28, 28, 1)         0         
_________________________________________________________________
encoder_model (Model)        (None, 100)               11457828  
_________________________________________________________________
model_decoder (Model)        (None, 28, 28, 1)         11497729  
Total params: 22,955,557
Trainable params: 22,955,557
Non-trainable params: 0
_________________________________________________________________


In [55]:
autoencoder.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss = 'mse', 
    metrics = ['accuracy']
)

In [56]:
h = autoencoder.fit(train_x, train_x, batch_size=1000, epochs = 100, callbacks=[ModelCheckpoint('checkpoint.h5', save_best_only=True)], validation_data=(test_x, test_x))

Train on 60000 samples, validate on 10000 samples
Epoch 1/100
28000/60000 [=============>................] - ETA: 4:33 - loss: 0.1164 - accuracy: 0.8070

KeyboardInterrupt: ignored

In [ ]:
def plot(h):
    plt.style.use('seaborn-darkgrid')

    epochs = h.epoch

    acc = h.history['accuracy']
    val_acc = h.history['val_accuracy']

    loss = h.history['loss']
    val_loss = h.history['val_loss']

    fig = plt.figure(figsize = (10,10),dpi = 100)
    ax = fig.add_axes([1,1,1,1])

    ax.plot(epochs,acc,label = 'Accuracy')
    ax.plot(epochs,val_acc,label = 'Val Accuracy')

    ax.plot(epochs,loss,label = 'Loss')
    ax.plot(epochs,val_loss,label = 'Val Loss')
    
    ax.set(xlabel = 'Number of Epochs',ylabel = 'Measure',title = 'Learning Curve')

    ax.legend()

In [ ]:
plot(h)

In [ ]:
 autoencoder.fit(train_x, train_x, batch_size=1000, epochs = 50, callbacks=[ModelCheckpoint('checkpoint.h5', save_best_only=True)], validation_data=(test_x, test_x))

In [ ]:
train_x.shape

In [ ]:
model_en.compile(
    optimizer='adam',
    loss = 'mse'
)

In [ ]:
#model_en.fit(train_x, train_y1, batch_size = 1000, epochs = 20)

In [ ]:
train_y.shape

(60000,)

In [ ]:
train_y1 = to_categorical(train_y)

In [ ]:
train_y1.shape

(60000, 10)